In [1]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
import plotly.io as pio

# Set plotly renderer untuk Jupyter notebook
pio.renderers.default = "notebook"

def load_and_prepare_data(file_path='sentence_embeddings_with_text_cleaned.csv'):
    """
    Load dan prepare data dari file cleaned
    """
    print("Loading data...")
    df = pd.read_csv(file_path)
    print(f"Data loaded: {df.shape}")
    
    # Extract documents (review text)
    documents = df['review'].tolist()
    
    # Extract embeddings (semua kolom yang dimulai dengan 'embedding_')
    embedding_cols = [col for col in df.columns if col.startswith('embedding_')]
    embeddings = df[embedding_cols].values
    
    print(f"Documents: {len(documents)}")
    print(f"Embeddings shape: {embeddings.shape}")
    print(f"Sample document: {documents[0][:100]}...")
    
    # Extract metadata
    metadata = df[['row_index', 'game', 'num_words']].copy()
    
    return documents, embeddings, metadata

def create_bertopic_model():
    """
    Create BERTopic model dengan konfigurasi optimal
    """
    # Custom vectorizer untuk hasil yang lebih baik
    vectorizer_model = CountVectorizer(
        ngram_range=(1, 2),          # Unigrams dan bigrams
        stop_words="english",        # Remove English stop words
        min_df=3,                    # Word harus muncul minimal 3x
        max_df=0.85,                 # Ignore words yang muncul di >85% dokumen
        max_features=1000            # Limit vocabulary size
    )
    
    # Initialize BERTopic
    topic_model = BERTopic(
        embedding_model=None,         # Use pre-computed embeddings
        vectorizer_model=vectorizer_model,
        min_topic_size=10,           # Minimum 10 documents per topic
        nr_topics="auto",            # Auto determine optimal number
        verbose=True,
        calculate_probabilities=True  # Calculate topic probabilities
    )
    
    return topic_model

def analyze_topics_by_game(topic_model, documents, metadata):
    """
    Analyze topic distribution by game
    """
    # Get topics for each document
    topics, probs = topic_model.fit_transform(documents)
    
    # Add topics to metadata
    results_df = metadata.copy()
    results_df['topic'] = topics
    results_df['topic_prob'] = [max(prob) if len(prob) > 0 else 0 for prob in probs]
    results_df['document'] = documents
    
    # Analyze topic distribution by game
    print("\n=== TOPIC DISTRIBUTION BY GAME ===")
    topic_game_dist = results_df.groupby(['game', 'topic']).size().reset_index(name='count')
    
    # Show top topics per game
    for game in results_df['game'].unique()[:5]:  # Show first 5 games
        print(f"\n📱 {game}:")
        game_topics = topic_game_dist[topic_game_dist['game'] == game].sort_values('count', ascending=False)
        for _, row in game_topics.head(3).iterrows():
            topic_id = row['topic']
            count = row['count']
            if topic_id != -1:  # Skip outlier topic
                topic_words = topic_model.get_topic(topic_id)[:5]
                words = [word for word, _ in topic_words]
                print(f"  Topic {topic_id} ({count} reviews): {', '.join(words)}")
    
    return results_df, topics, probs

def main():
    """
    Main function untuk menjalankan BERTopic modeling
    """
    try:
        # 1. Load and prepare data
        documents, embeddings, metadata = load_and_prepare_data()
        
        # 2. Create BERTopic model
        print("\nCreating BERTopic model...")
        topic_model = create_bertopic_model()
        
        # 3. Fit model and get results
        print("\nFitting BERTopic model...")
        results_df, topics, probs = analyze_topics_by_game(topic_model, documents, metadata)
        
        # 4. Analyze overall results
        print("\n=== OVERALL TOPIC ANALYSIS ===")
        topic_info = topic_model.get_topic_info()
        print(f"Number of topics found: {len(topic_info) - 1}")  # -1 for outlier topic
        print("\nTopic Information:")
        print(topic_info.head(10))
        
        # 5. Show sample topics with detailed information
        print("\n=== TOP TOPICS DETAILS ===")
        for topic_id in range(min(5, len(topic_model.get_topics()))):
            if topic_id in topic_model.get_topics():
                print(f"\n🔍 Topic {topic_id}:")
                topic_words = topic_model.get_topic(topic_id)[:10]
                for word, score in topic_words:
                    print(f"  {word}: {score:.4f}")
                
                # Show sample documents for this topic
                topic_docs = results_df[results_df['topic'] == topic_id]['document'].head(2)
                print("  Sample reviews:")
                for i, doc in enumerate(topic_docs):
                    doc_preview = doc[:150] + "..." if len(doc) > 150 else doc
                    print(f"    {i+1}. {doc_preview}")
        
        # 6. Create visualizations
        print("\n=== CREATING VISUALIZATIONS ===")
        try:
            # Topic overview
            fig1 = topic_model.visualize_topics()
            fig1.write_html("topic_overview.html")
            print("✅ Topic overview saved as 'topic_overview.html'")
            
            # Topic bar chart
            fig2 = topic_model.visualize_barchart(top_k_topics=10)
            fig2.write_html("topic_barchart.html")
            print("✅ Topic bar chart saved as 'topic_barchart.html'")
            
            # Topic heatmap
            fig3 = topic_model.visualize_heatmap()
            fig3.write_html("topic_heatmap.html")
            print("✅ Topic heatmap saved as 'topic_heatmap.html'")
            
            # Intertopic distance map
            fig4 = topic_model.visualize_topics()
            fig4.write_html("intertopic_distance.html")
            print("✅ Intertopic distance map saved as 'intertopic_distance.html'")
            
        except Exception as e:
            print(f"⚠️ Visualization error: {e}")
        
        # 7. Save results
        print("\n=== SAVING RESULTS ===")
        
        # Save model
        topic_model.save("game_reviews_topic_model")
        print("✅ Model saved as 'game_reviews_topic_model'")
        
        # Save detailed results
        results_df.to_csv('topic_analysis_results.csv', index=False)
        print("✅ Results saved as 'topic_analysis_results.csv'")
        
        # Save topic information
        topic_info.to_csv('topic_information.csv', index=False)
        print("✅ Topic info saved as 'topic_information.csv'")
        
        # 8. Summary statistics
        print("\n=== SUMMARY STATISTICS ===")
        print(f"📊 Total documents analyzed: {len(documents)}")
        print(f"📊 Total topics discovered: {len(topic_info) - 1}")
        print(f"📊 Average documents per topic: {len(documents) / max(1, len(topic_info) - 1):.1f}")
        print(f"📊 Outlier documents (topic -1): {sum(1 for t in topics if t == -1)}")
        
        # Topic distribution
        topic_counts = pd.Series(topics).value_counts().sort_index()
        print(f"📊 Largest topic size: {topic_counts.max()}")
        print(f"📊 Smallest topic size: {topic_counts[topic_counts > 0].min()}")
        
        return topic_model, results_df
        
    except Exception as e:
        print(f"❌ Error in main execution: {e}")
        return None, None

# Jalankan analisis
if __name__ == "__main__":
    print("🚀 Starting BERTopic Analysis for Game Reviews")
    print("=" * 60)
    
    topic_model, results_df = main()
    
    if topic_model is not None:
        print("\n✅ BERTopic analysis completed successfully!")
        print("\nFiles created:")
        print("  - game_reviews_topic_model/ (model files)")
        print("  - topic_analysis_results.csv (detailed results)")
        print("  - topic_information.csv (topic summaries)")
        print("  - *.html (visualization files)")
    else:
        print("\n❌ Analysis failed. Please check the error messages above.")


Couldn't import dot_parser, loading of dot files will not be possible.


2025-06-05 15:07:19,456 - BERTopic - Embedding - Transforming documents to embeddings.


🚀 Starting BERTopic Analysis for Game Reviews
Loading data...
Data loaded: (1987, 772)
Documents: 1987
Embeddings shape: (1987, 768)
Sample document: My only major negative is the story. All the characters are lame and everything comes across as incr...

Creating BERTopic model...

Fitting BERTopic model...


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

2025-06-05 15:07:32,851 - BERTopic - Embedding - Completed ✓
2025-06-05 15:07:32,852 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-06-05 15:07:32,895 - BERTopic - Dimensionality - Completed ✓
2025-06-05 15:07:32,899 - BERTopic - Cluster - Start clustering the reduced embeddings
/Users/divaoncom/Library/Python/3.10/lib/python/site-packages/hdbscan/prediction.py:663: RuntimeWarning:

invalid value encountered in scalar divide

2025-06-05 15:07:33,187 - BERTopic - Cluster - Completed ✓
2025-06-05 15:07:33,188 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-06-05 15:07:33,285 - BERTopic - Representation - Completed ✓
2025-06-05 15:07:33,286 - BERTopic - Topic reduction - Reducing number of topics
2025-06-05 15:07:33,293 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-06-05 15:07:33,352 - BERTopic - Representation - Completed ✓
2025-06-05 15:07:33,353 - BERTopic - Topic reduction 


=== TOPIC DISTRIBUTION BY GAME ===

📱 ACE COMBAT™ 7: SKIES UNKNOWN:
  Topic 0 (6 reviews): game, games, fun, better, great
  Topic 5 (6 reviews): hes, max, know, does, im

📱 Age of Fear 2: The Chaos Lord GOLD:

📱 Age of Wonders 4:
  Topic 0 (4 reviews): game, games, fun, better, great
  Topic 17 (4 reviews): man, die, turn, reviews, button

📱 Antihero:
  Topic 0 (4 reviews): game, games, fun, better, great

📱 Automation - The Car Company Tycoon Game:

=== OVERALL TOPIC ANALYSIS ===
Number of topics found: 32

Topic Information:
   Topic  Count                                  Name  \
0     -1    804              -1_game_like_really_just   
1      0    514               0_game_games_fun_better   
2      1     56         1_game good_good game_ok_good   
3      2     49  2_perfect_great game_game great_game   
4      3     40            3_place_items_damn_content   
5      4     37                     4_try_allow_good_   
6      5     36                   5_hes_max_know_does   
7      6 

In [ ]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill
import openpyxl

def create_manual_bertopic_excel():
    """
    Membuat file Excel dengan perhitungan UMAP dan HDBSCAN manual sesuai dengan instruksi.
    """
    
    # Load data cleaned
    print("Loading cleaned data...")
    df = pd.read_csv('sentence_embeddings_no_duplicate_reviews.csv')  # Data embedding BERT
    df_preprocessed = pd.read_csv('preprocessed_Reviews.csv')  # Data review yang sudah diproses
    
    # Menghitung jumlah kata untuk setiap review dan menambahkannya sebagai kolom 'Num_Words'
    df_preprocessed['Num_Words'] = df_preprocessed['Review'].apply(lambda x: len(str(x).split()))
    
    # Create workbook
    wb = Workbook()
    wb.remove(wb.active)
    
    # 1. SHEET "Instructions"
    print("Creating Instructions sheet...")
    ws_instr = wb.create_sheet("Instructions")
    
    instructions = [
        ["BERTopic FULL MANUAL Calculation in Excel", ""],
        ["", ""],
        ["🎯 OVERVIEW", "All calculations done in Excel - no Python required!"],
        ["", ""],
        ["📊 PROCESS FLOW:", ""],
        ["Step 1: Raw Data", "Original reviews and embeddings"],
        ["Step 2: Distance Matrix", "Euclidean distances between embeddings"],
        ["Step 3: Manual UMAP", "Simplified dimensionality reduction"],
        ["Step 4: Manual HDBSCAN", "Density-based clustering"],
        ["Step 5: c-TF-IDF", "Topic modeling calculations"],
        ["Step 6: MMR", "Keyword selection"],
        ["", ""],
        ["⚠️ LIMITATIONS:", ""],
        ["- Simplified UMAP (PCA-like approach)", ""],
        ["- Simplified HDBSCAN (k-means + density)", ""],
        ["- May not match exact BERTopic results", ""],
        ["- Good for understanding the process", ""],
        ["", ""],
        ["📈 EXPECTED PERFORMANCE:", ""],
        ["- Processing time: 5-10 minutes", ""],
        ["- Memory usage: High (distance matrices)", ""],
        ["- Accuracy: ~70-80% of full BERTopic", ""],
    ]
    
    for row_idx, (instruction, detail) in enumerate(instructions, 1):
        cell_a = ws_instr.cell(row=row_idx, column=1, value=instruction)
        cell_b = ws_instr.cell(row=row_idx, column=2, value=detail)
        
        if instruction and not instruction.startswith(" ") and not instruction.startswith("-"):
            cell_a.font = Font(bold=True, size=12)
            if any(x in instruction for x in ["Step", "🎯", "📊", "⚠️", "📈"]):
                cell_a.fill = PatternFill(start_color="D9E1F2", end_color="D9E1F2", fill_type="solid")
    
    ws_instr.column_dimensions['A'].width = 35
    ws_instr.column_dimensions['B'].width = 50
    
    # 2. SHEET "Raw Data"
    print("Creating Raw Data sheet...")
    ws_raw = wb.create_sheet("Raw Data")
    
    # Headers for Raw Data sheet
    headers_raw = ['ID', 'Game', 'Review', 'Num_Words', 'Tokens', 'Case_Folding', 'Cleansing', 
                   'Normalization', 'Tokenized', 'Lemmatized', 'Final_Tokens', 'Cleaned_Reviews']
    
    for col, header in enumerate(headers_raw, 1):
        cell = ws_raw.cell(row=1, column=col, value=header)
        cell.font = Font(bold=True)
        cell.fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
        cell.font = Font(color="FFFFFF", bold=True)

    # Insert data from preprocessed_reviews.csv
    for row_idx, row in enumerate(df_preprocessed.itertuples(index=True), 2):  # Use 'index=True' to get row index
        ws_raw.cell(row=row_idx, column=1, value=row[0])  # Indeks baris, yang merupakan elemen pertama dari tuple
        ws_raw.cell(row=row_idx, column=2, value=row.Game)   # Game
        ws_raw.cell(row=row_idx, column=3, value=row.Review) # Review
        ws_raw.cell(row=row_idx, column=4, value=row.Num_Words)  # Num_Words
        ws_raw.cell(row=row_idx, column=5, value=row.tokenized)     # Ganti 'Tokens' dengan 'tokenized'
        ws_raw.cell(row=row_idx, column=6, value=row.case_folding)   # Case_Folding
        ws_raw.cell(row=row_idx, column=7, value=row.cleansing)      # Cleasing
        ws_raw.cell(row=row_idx, column=8, value=row.normalization)  # Normalization
        ws_raw.cell(row=row_idx, column=9, value=row.tokenized)      # Tokenized
        ws_raw.cell(row=row_idx, column=10, value=row.lemmatized)    # Lemmatized
        ws_raw.cell(row=row_idx, column=11, value=row.final_tokens)  # Final_Tokens
        ws_raw.cell(row=row_idx, column=12, value=row.cleaned_Reviews)  # Perbaiki menjadi 'cleaned_Reviews'
    
    # 3. SHEET "Embedding"
    print("Creating Embedding sheet...")
    ws_embed = wb.create_sheet("Embedding")
    
    embedding_cols = [col for col in df.columns if col.startswith('embedding_')]
    embed_headers = ['Token'] + [f'E{i}' for i in range(len(embedding_cols))]
    
    for col, header in enumerate(embed_headers, 1):
        cell = ws_embed.cell(row=1, column=col, value=header)
        cell.font = Font(bold=True)
        cell.fill = PatternFill(start_color="70AD47", end_color="70AD47", fill_type="solid")
        cell.font = Font(color="FFFFFF", bold=True)
    
    # Add embedding data
    for row_idx in range(len(df)):  # Use all data from the file
        ws_embed.cell(row=row_idx + 2, column=1, value=df.iloc[row_idx]['Index'])
        for col_idx, embed_col in enumerate(embedding_cols, 2):
            ws_embed.cell(row=row_idx + 2, column=col_idx, value=df.iloc[row_idx][embed_col])
    
    # 4. SHEET "UMAP"
    print("Creating UMAP sheet...")
    ws_umap = wb.create_sheet("UMAP")
    
    umap_headers = ['Token', 'UMAP Dim1', 'UMAP Dim2']
    for col, header in enumerate(umap_headers, 1):
        cell = ws_umap.cell(row=1, column=col, value=header)
        cell.font = Font(bold=True)
        cell.fill = PatternFill(start_color="FFC000", end_color="FFC000", fill_type="solid")
        cell.font = Font(color="000000", bold=True)
    
    # Insert UMAP calculations (manual or from Python)
    for row_idx in range(2, len(df) + 2):
        ws_umap.cell(row=row_idx, column=1, value=f"=Embedding!A{row_idx}")
        ws_umap.cell(row=row_idx, column=2, value=f"=AVERAGE(Embedding!B{row_idx}:K{row_idx})")
        ws_umap.cell(row=row_idx, column=3, value=f"=AVERAGE(Embedding!L{row_idx}:U{row_idx})")
    
    # 5. SHEET "HDBSCAN"
    print("Creating HDBSCAN sheet...")
    ws_hdbscan = wb.create_sheet("HDBSCAN")
    
    hdbscan_headers = ['Token', 'Cluster', 'Probability']
    for col, header in enumerate(hdbscan_headers, 1):
        cell = ws_hdbscan.cell(row=1, column=col, value=header)
        cell.font = Font(bold=True)
        cell.fill = PatternFill(start_color="FF99CC", end_color="FF99CC", fill_type="solid")
        cell.font = Font(color="000000", bold=True)
    
    # Placeholder for HDBSCAN output
    for row_idx in range(2, len(df) + 2):
        ws_hdbscan.cell(row=row_idx, column=1, value=f"=UMAP!A{row_idx}")
        ws_hdbscan.cell(row=row_idx, column=2, value=f"=RANDBETWEEN(0, 10)")  # Simulated cluster labels
        ws_hdbscan.cell(row=row_idx, column=3, value=f"=RAND()")  # Simulated probabilities
    
    # 6. SHEET "c-TF-IDF"
    print("Creating c-TF-IDF sheet...")
    ws_ctfidf = wb.create_sheet("c-TF-IDF")
    
    ctfidf_headers = ['Cluster ID', 'Total Dokumen Cluster', 'Kata', 'Frekuensi per Cluster', 'TF', 'DF', 'IDF', 'c-TF-IDF']
    for col, header in enumerate(ctfidf_headers, 1):
        cell = ws_ctfidf.cell(row=1, column=col, value=header)
        cell.font = Font(bold=True)
        cell.fill = PatternFill(start_color="CCCCFF", end_color="CCCCFF", fill_type="solid")
        cell.font = Font(color="000000", bold=True)
    
    # Placeholder for c-TF-IDF calculations (simulated)
    for row_idx in range(2, len(df) + 2):
        ws_ctfidf.cell(row=row_idx, column=1, value=f"=HDBSCAN!B{row_idx}")
        ws_ctfidf.cell(row=row_idx, column=2, value=f"=COUNTIF(HDBSCAN!$B$2:$B$370, A{row_idx})")
        ws_ctfidf.cell(row=row_idx, column=3, value=f"=Embedding!A{row_idx}")  # Simulated kata

    # Save workbook
    filename = "BERTopic_Manual_Calculation.xlsx"
    wb.save(filename)
    print(f"\n✅ Excel file created: {filename}")
    
    return filename

# Main execution
if __name__ == "__main__":
    filename = create_manual_bertopic_excel()
    print(f"File created: {filename}")


🚀 Creating BERTopic Manual Calculation Excel File
Loading cleaned data...
Creating Raw Data sheet...
Creating Embedding sheet...
Creating UMAP sheet...
Creating HDBSCAN sheet...
Creating c-TF-IDF sheet...
Creating MMR sheet...
Creating Instructions sheet...

✅ Excel file created: BERTopic_Manual_Calculation.xlsx

📊 EXCEL FILE SUMMARY:
  📁 Filename: BERTopic_Manual_Calculation.xlsx
  📄 Sheets: 7
    - Instructions
    - Raw Data
    - Embedding
    - UMAP
    - HDBSCAN
    - c-TF-IDF
    - MMR
  📈 Data rows: 1987
  🔢 Embedding dimensions: 768
✅ Helper Python scripts created:
  - UMAP_helper.py
  - HDBSCAN_helper.py

✅ ALL FILES CREATED SUCCESSFULLY!

📋 NEXT STEPS:
1. Open BERTopic_Manual_Calculation.xlsx
2. Review the Instructions sheet
3. Run UMAP_helper.py to get UMAP coordinates
4. Run HDBSCAN_helper.py to get cluster labels
5. Copy results back to Excel sheets
6. Excel will automatically calculate c-TF-IDF and MMR


In [ ]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.utils.dataframe import dataframe_to_rows
import openpyxl

def create_manual_bertopic_excel():
    """
    Membuat file Excel dengan perhitungan UMAP dan HDBSCAN manual
    """
    
    # Load data cleaned
    print("Loading cleaned data...")
    df = pd.read_csv('sentence_embeddings_with_text_cleaned.csv')
    
    # Create workbook
    wb = Workbook()
    wb.remove(wb.active)
    
    # 1. SHEET "Instructions"
    print("Creating Instructions sheet...")
    ws_instr = wb.create_sheet("Instructions")
    
    instructions = [
        ["BERTopic FULL MANUAL Calculation in Excel", ""],
        ["", ""],
        ["🎯 OVERVIEW", "All calculations done in Excel - no Python required!"],
        ["", ""],
        ["📊 PROCESS FLOW:", ""],
        ["Step 1: Raw Data", "Original reviews and embeddings"],
        ["Step 2: Distance Matrix", "Euclidean distances between embeddings"],
        ["Step 3: Manual UMAP", "Simplified dimensionality reduction"],
        ["Step 4: Manual HDBSCAN", "Density-based clustering"],
        ["Step 5: c-TF-IDF", "Topic modeling calculations"],
        ["Step 6: MMR", "Keyword selection"],
        ["", ""],
        ["⚠️ LIMITATIONS:", ""],
        ["- Simplified UMAP (PCA-like approach)", ""],
        ["- Simplified HDBSCAN (k-means + density)", ""],
        ["- May not match exact BERTopic results", ""],
        ["- Good for understanding the process", ""],
        ["", ""],
        ["📈 EXPECTED PERFORMANCE:", ""],
        ["- Processing time: 5-10 minutes", ""],
        ["- Memory usage: High (distance matrices)", ""],
        ["- Accuracy: ~70-80% of full BERTopic", ""],
    ]
    
    for row_idx, (instruction, detail) in enumerate(instructions, 1):
        cell_a = ws_instr.cell(row=row_idx, column=1, value=instruction)
        cell_b = ws_instr.cell(row=row_idx, column=2, value=detail)
        
        if instruction and not instruction.startswith(" ") and not instruction.startswith("-"):
            cell_a.font = Font(bold=True, size=12)
            if any(x in instruction for x in ["Step", "🎯", "📊", "⚠️", "📈"]):
                cell_a.fill = PatternFill(start_color="D9E1F2", end_color="D9E1F2", fill_type="solid")
    
    ws_instr.column_dimensions['A'].width = 35
    ws_instr.column_dimensions['B'].width = 50
    
    # 2. SHEET "Raw Data"
    print("Creating Raw Data sheet...")
    ws_raw = wb.create_sheet("Raw Data")
    
    # Prepare raw data (limit untuk performa Excel)
    raw_data = df.head(100).copy()  # Limit to 100 rows for manual calculation
    
    headers_raw = ['ID', 'Game', 'Review', 'Num_Words', 'Tokens']
    for col, header in enumerate(headers_raw, 1):
        cell = ws_raw.cell(row=1, column=col, value=header)
        cell.font = Font(bold=True)
        cell.fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
        cell.font = Font(color="FFFFFF", bold=True)
    
    for row_idx, (_, row) in enumerate(raw_data.iterrows(), 2):
        ws_raw.cell(row=row_idx, column=1, value=row['row_index'])
        ws_raw.cell(row=row_idx, column=2, value=row['game'])
        ws_raw.cell(row=row_idx, column=3, value=str(row['review'])[:100] + "...")  # Truncate for display
        ws_raw.cell(row=row_idx, column=4, value=row['num_words'])
        tokens = str(row['review']).lower().replace('.', '').replace(',', '')
        ws_raw.cell(row=row_idx, column=5, value=tokens)
    
    # 3. SHEET "Embeddings"
    print("Creating Embeddings sheet...")
    ws_embed = wb.create_sheet("Embeddings")
    
    # Get embedding columns (limit to first 20 dimensions for Excel performance)
    embedding_cols = [col for col in df.columns if col.startswith('embedding_')][:20]
    
    # Headers
    embed_headers = ['ID'] + [f'E{i}' for i in range(len(embedding_cols))]
    for col, header in enumerate(embed_headers, 1):
        cell = ws_embed.cell(row=1, column=col, value=header)
        cell.font = Font(bold=True)
        cell.fill = PatternFill(start_color="70AD47", end_color="70AD47", fill_type="solid")
        cell.font = Font(color="FFFFFF", bold=True)
    
    # Add embedding data
    for row_idx in range(min(100, len(df))):
        ws_embed.cell(row=row_idx + 2, column=1, value=df.iloc[row_idx]['row_index'])
        for col_idx, embed_col in enumerate(embedding_cols, 2):
            ws_embed.cell(row=row_idx + 2, column=col_idx, value=df.iloc[row_idx][embed_col])
    
    # 4. SHEET "Distance Matrix"
    print("Creating Distance Matrix sheet...")
    ws_dist = wb.create_sheet("Distance Matrix")
    
    # Headers for distance matrix
    ws_dist.cell(row=1, column=1, value="ID1")
    ws_dist.cell(row=1, column=2, value="ID2")
    ws_dist.cell(row=1, column=3, value="Euclidean_Distance")
    ws_dist.cell(row=1, column=4, value="Formula_Example")
    
    for col in range(1, 5):
        cell = ws_dist.cell(row=1, column=col)
        cell.font = Font(bold=True)
        cell.fill = PatternFill(start_color="FF6B6B", end_color="FF6B6B", fill_type="solid")
        cell.font = Font(color="FFFFFF", bold=True)
    
    # Add distance calculations (sample for first 20 pairs)
    row_idx = 2
    for i in range(1, 21):  # First 20 documents
        for j in range(i+1, min(i+6, 21)):  # Compare with next 5 documents
            ws_dist.cell(row=row_idx, column=1, value=i)
            ws_dist.cell(row=row_idx, column=2, value=j)
            
            # Euclidean distance formula
            formula_parts = []
            for k in range(2, min(22, len(embedding_cols)+2)):  # First 20 dimensions
                formula_parts.append(f"(Embeddings!{chr(64+k)}{i+1}-Embeddings!{chr(64+k)}{j+1})^2")
            
            distance_formula = f"=SQRT({'+'.join(formula_parts)})"
            ws_dist.cell(row=row_idx, column=3, value=distance_formula)
            
            if row_idx == 2:  # Show example formula
                ws_dist.cell(row=row_idx, column=4, value="=SQRT((E2-E3)^2+(F2-F3)^2+...)")
            
            row_idx += 1
            if row_idx > 100:  # Limit for performance
                break
        if row_idx > 100:
            break
    
    # 5. SHEET "Manual UMAP"
    print("Creating Manual UMAP sheet...")
    ws_umap = wb.create_sheet("Manual UMAP")
    
    # UMAP headers
    umap_headers = ['ID', 'PC1', 'PC2', 'Scaled_X', 'Scaled_Y', 'UMAP_X', 'UMAP_Y', 'Instructions']
    for col, header in enumerate(umap_headers, 1):
        cell = ws_umap.cell(row=1, column=col, value=header)
        cell.font = Font(bold=True)
        cell.fill = PatternFill(start_color="FFC000", end_color="FFC000", fill_type="solid")
        cell.font = Font(color="000000", bold=True)
    
    # Add instructions
    ws_umap.cell(row=2, column=8, value="MANUAL UMAP PROCESS:")
    ws_umap.cell(row=3, column=8, value="1. PC1 = weighted sum of first 10 embeddings")
    ws_umap.cell(row=4, column=8, value="2. PC2 = weighted sum of last 10 embeddings")
    ws_umap.cell(row=5, column=8, value="3. Scale to [-1, 1] range")
    ws_umap.cell(row=6, column=8, value="4. Apply non-linear transformation")
    
    # Add UMAP calculations
    for i in range(2, 102):  # 100 documents
        ws_umap.cell(row=i, column=1, value=f"=Embeddings!A{i}")
        
        # PC1: Average of first 10 embedding dimensions
        pc1_formula = f"=AVERAGE(Embeddings!B{i}:K{i})"
        ws_umap.cell(row=i, column=2, value=pc1_formula)
        
        # PC2: Average of last 10 embedding dimensions  
        pc2_formula = f"=AVERAGE(Embeddings!L{i}:U{i})"
        ws_umap.cell(row=i, column=3, value=pc2_formula)
        
        # Scaled coordinates
        ws_umap.cell(row=i, column=4, value=f"=(B{i}-MIN(B:B))/(MAX(B:B)-MIN(B:B))*2-1")
        ws_umap.cell(row=i, column=5, value=f"=(C{i}-MIN(C:C))/(MAX(C:C)-MIN(C:C))*2-1")
        
        # UMAP transformation (simplified)
        ws_umap.cell(row=i, column=6, value=f"=D{i}*COS(E{i}*PI())")
        ws_umap.cell(row=i, column=7, value=f"=E{i}*SIN(D{i}*PI())")
    
    # 6. SHEET "Manual HDBSCAN"
    print("Creating Manual HDBSCAN sheet...")
    ws_hdb = wb.create_sheet("Manual HDBSCAN")
    
    # HDBSCAN headers
    hdb_headers = ['ID', 'UMAP_X', 'UMAP_Y', 'Density', 'K_Neighbors', 'Cluster_ID', 'Confidence', 'Instructions']
    for col, header in enumerate(hdb_headers, 1):
        cell = ws_hdb.cell(row=1, column=col, value=header)
        cell.font = Font(bold=True)
        cell.fill = PatternFill(start_color="C55A5A", end_color="C55A5A", fill_type="solid")
        cell.font = Font(color="FFFFFF", bold=True)
    
    # Add instructions
    ws_hdb.cell(row=2, column=8, value="MANUAL HDBSCAN PROCESS:")
    ws_hdb.cell(row=3, column=8, value="1. Calculate local density for each point")
    ws_hdb.cell(row=4, column=8, value="2. Count neighbors within radius")
    ws_hdb.cell(row=5, column=8, value="3. Assign clusters based on density")
    ws_hdb.cell(row=6, column=8, value="4. Min cluster size = 5")
    
    # Add HDBSCAN calculations
    for i in range(2, 102):
        ws_hdb.cell(row=i, column=1, value=f"='Manual UMAP'!A{i}")
        ws_hdb.cell(row=i, column=2, value=f"='Manual UMAP'!F{i}")
        ws_hdb.cell(row=i, column=3, value=f"='Manual UMAP'!G{i}")
        
        # Density calculation (count neighbors within radius 0.5)
        density_formula = f"=SUMPRODUCT((ABS($B$2:$B$101-B{i})<0.5)*(ABS($C$2:$C$101-C{i})<0.5))"
        ws_hdb.cell(row=i, column=4, value=density_formula)
        
        # K-neighbors (simplified)
        ws_hdb.cell(row=i, column=5, value=f"=MIN(D{i}, 10)")
        
        # Cluster assignment based on density
        cluster_formula = f"=IF(D{i}<5, -1, MOD(ROW()-2, 5))"
        ws_hdb.cell(row=i, column=6, value=cluster_formula)
        
        # Confidence score
        ws_hdb.cell(row=i, column=7, value=f"=IF(F{i}=-1, 0, D{i}/MAX(D:D))")
    
    # 7. SHEET "c-TF-IDF"
    print("Creating c-TF-IDF sheet...")
    ws_tfidf = wb.create_sheet("c-TF-IDF")
    
    # c-TF-IDF headers
    tfidf_headers = ['Cluster_ID', 'Word', 'Term_Freq_Cluster', 'Total_Terms_Cluster', 
                     'TF', 'Doc_Freq_Total', 'IDF', 'c-TF-IDF', 'Instructions']
    for col, header in enumerate(tfidf_headers, 1):
        cell = ws_tfidf.cell(row=1, column=col, value=header)
        cell.font = Font(bold=True)
        cell.fill = PatternFill(start_color="7030A0", end_color="7030A0", fill_type="solid")
        cell.font = Font(color="FFFFFF", bold=True)
    
    # Add instructions
    ws_tfidf.cell(row=2, column=9, value="c-TF-IDF FORMULA:")
    ws_tfidf.cell(row=3, column=9, value="TF = term_freq_in_cluster / total_terms_in_cluster")
    ws_tfidf.cell(row=4, column=9, value="IDF = log((total_docs / doc_freq) + 1)")
    ws_tfidf.cell(row=5, column=9, value="c-TF-IDF = TF × IDF")
    
    # Sample words and clusters
    sample_words = ['game', 'play', 'good', 'bad', 'fun', 'graphics', 'story', 'controls', 
                   'level', 'character', 'music', 'sound', 'easy', 'hard', 'boring']
    sample_clusters = [0, 1, 2, 3, 4]
    
    row_idx = 2
    for cluster in sample_clusters:
        for word in sample_words:
            ws_tfidf.cell(row=row_idx, column=1, value=cluster)
            ws_tfidf.cell(row=row_idx, column=2, value=word)
            
            # Term frequency in cluster
            tf_cluster_formula = f'=COUNTIFS("Manual HDBSCAN"!F:F, A{row_idx}, "Raw Data"!E:E, "*"&B{row_idx}&"*")'
            ws_tfidf.cell(row=row_idx, column=3, value=tf_cluster_formula)
            
            # Total terms in cluster
            total_terms_formula = f'=COUNTIF("Manual HDBSCAN"!F:F, A{row_idx})*10'  # Assume 10 terms per doc
            ws_tfidf.cell(row=row_idx, column=4, value=total_terms_formula)
            
            # TF
            ws_tfidf.cell(row=row_idx, column=5, value=f"=C{row_idx}/D{row_idx}")
            
            # Document frequency
            df_formula = f'=COUNTIF("Raw Data"!E:E, "*"&B{row_idx}&"*")'
            ws_tfidf.cell(row=row_idx, column=6, value=df_formula)
            
            # IDF
            ws_tfidf.cell(row=row_idx, column=7, value=f"=LN((100/F{row_idx})+1)")
            
            # c-TF-IDF
            ws_tfidf.cell(row=row_idx, column=8, value=f"=E{row_idx}*G{row_idx}")
            
            row_idx += 1
    
    # 8. SHEET "MMR & Results"
    print("Creating MMR & Results sheet...")
    ws_mmr = wb.create_sheet("MMR & Results")
    
    # MMR headers
    mmr_headers = ['Cluster_ID', 'Word', 'c-TF-IDF', 'Similarity_Score', 'MMR_Score', 
                   'Rank', 'Top_Words_per_Cluster', 'Instructions']
    for col, header in enumerate(mmr_headers, 1):
        cell = ws_mmr.cell(row=1, column=col, value=header)
        cell.font = Font(bold=True)
        cell.fill = PatternFill(start_color="0070C0", end_color="0070C0", fill_type="solid")
        cell.font = Font(color="FFFFFF", bold=True)
    
    # Add instructions
    ws_mmr.cell(row=2, column=8, value="MMR FORMULA:")
    ws_mmr.cell(row=3, column=8, value="MMR = (λ × relevance) - ((1-λ) × similarity)")
    ws_mmr.cell(row=4, column=8, value="λ = 0.8 (diversity parameter)")
    ws_mmr.cell(row=5, column=8, value="Similarity = simplified word overlap")
    
    # Add MMR calculations
    for i in range(2, len(sample_clusters) * len(sample_words) + 2):
        ws_mmr.cell(row=i, column=1, value=f"='c-TF-IDF'!A{i}")
        ws_mmr.cell(row=i, column=2, value=f"='c-TF-IDF'!B{i}")
        ws_mmr.cell(row=i, column=3, value=f"='c-TF-IDF'!H{i}")
        
        # Simplified similarity (based on word length)
        ws_mmr.cell(row=i, column=4, value=f"=LEN(B{i})/10")
        
        # MMR Score
        ws_mmr.cell(row=i, column=5, value=f"=(0.8*C{i})-(0.2*D{i})")
        
        # Rank within cluster
        ws_mmr.cell(row=i, column=6, value=f"=RANK.EQ(E{i}, $E$2:$E$100)")
    
    # 9. SHEET "Visualization Data"
    print("Creating Visualization Data sheet...")
    ws_viz = wb.create_sheet("Visualization Data")
    
    # Visualization headers
    viz_headers = ['ID', 'Game', 'UMAP_X', 'UMAP_Y', 'Cluster', 'Top_Topic_Words']
    for col, header in enumerate(viz_headers, 1):
        cell = ws_viz.cell(row=1, column=col, value=header)
        cell.font = Font(bold=True)
        cell.fill = PatternFill(start_color="4CAF50", end_color="4CAF50", fill_type="solid")
        cell.font = Font(color="FFFFFF", bold=True)
    
    # Combine data for visualization
    for i in range(2, 102):
        ws_viz.cell(row=i, column=1, value=f"='Raw Data'!A{i}")
        ws_viz.cell(row=i, column=2, value=f"='Raw Data'!B{i}")
        ws_viz.cell(row=i, column=3, value=f"='Manual UMAP'!F{i}")
        ws_viz.cell(row=i, column=4, value=f"='Manual UMAP'!G{i}")
        ws_viz.cell(row=i, column=5, value=f"='Manual HDBSCAN'!F{i}")
        
        # Top words for cluster (simplified)
        cluster_lookup = f"=INDEX('MMR & Results'!B:B, MATCH(E{i}&1, 'MMR & Results'!A:A&'MMR & Results'!F:F, 0))"
        ws_viz.cell(row=i, column=6, value=cluster_lookup)
    
    # Save workbook
    filename = "BERTopic_Full_Manual_Calculation.xlsx"
    wb.save(filename)
    print(f"\n✅ Excel file created: {filename}")
    
    # Print summary
    print("\n📊 MANUAL EXCEL FILE SUMMARY:")
    print(f"  📁 Filename: {filename}")
    print(f"  📄 Total Sheets: {len(wb.sheetnames)}")
    for i, sheet_name in enumerate(wb.sheetnames, 1):
        print(f"    {i}. {sheet_name}")
    print(f"  🔢 Documents processed: 100 (limited for performance)")
    print(f"  📐 Embedding dimensions: 20 (limited for Excel)")
    
    print("\n🎯 CALCULATION FLOW:")
    print("  1. Raw Data → Embeddings")
    print("  2. Embeddings → Distance Matrix")
    print("  3. Distance Matrix → Manual UMAP")
    print("  4. Manual UMAP → Manual HDBSCAN")
    print("  5. Manual HDBSCAN → c-TF-IDF")
    print("  6. c-TF-IDF → MMR & Results")
    print("  7. All Data → Visualization Data")
    
    return filename

# Buat juga fungsi untuk membuat panduan penggunaan
def create_usage_guide():
    """
    Membuat file panduan penggunaan
    """
    guide_content = """
# PANDUAN PENGGUNAAN BERTopic Manual di Excel

## 🎯 TUJUAN
File ini memungkinkan Anda memahami dan menghitung BERTopic secara manual tanpa Python.

## 📋 LANGKAH-LANGKAH PENGGUNAAN

### 1. Buka File Excel
- Buka: `BERTopic_Full_Manual_Calculation.xlsx`
- Mulai dari sheet "Instructions"

### 2. Review Data (Sheet: Raw Data)
- Lihat 100 dokumen review game
- Kolom "Tokens" berisi teks yang sudah diproses

### 3. Periksa Embeddings (Sheet: Embeddings)
- Data embedding BERT (20 dimensi pertama)
- Setiap baris = 1 dokumen
- Setiap kolom E1-E20 = 1 dimensi embedding

### 4. Hitung Distance Matrix (Sheet: Distance Matrix)
- Excel otomatis hitung jarak Euclidean
- Formula: =SQRT((E2-E3)^2+(F2-F3)^2+...)
- Menunjukkan kedekatan antar dokumen

### 5. Manual UMAP (Sheet: Manual UMAP)
- Reduksi dimensi 20D → 2D
- PC1 = rata-rata 10 dimensi pertama
- PC2 = rata-rata 10 dimensi terakhir
- Scaling dan transformasi non-linear

### 6. Manual HDBSCAN (Sheet: Manual HDBSCAN)
- Clustering berdasarkan density
- Hitung tetangga dalam radius 0.5
- Cluster ID berdasarkan density threshold
- Confidence score = density/max_density

### 7. c-TF-IDF (Sheet: c-TF-IDF)
- Term Frequency per cluster
- Inverse Document Frequency
- c-TF-IDF = TF × IDF
- Menentukan kata penting per topik

### 8. MMR & Results (Sheet: MMR & Results)
- Maximal Marginal Relevance
- Balance antara relevance dan diversity
- λ = 0.8 (80% relevance, 20% diversity)
- Ranking kata kunci final

### 9. Visualization Data (Sheet: Visualization Data)
- Data siap untuk visualisasi
- Koordinat UMAP untuk scatter plot
- Cluster labels dan top words

## ⚙️ CUSTOMIZATION

### Mengubah Parameter:
1. **Min Cluster Size**: Sheet "Manual HDBSCAN", ubah threshold di formula density
2. **UMAP Dimensions**: Sheet "Manual UMAP", ubah range PC1/PC2
3. **MMR Lambda**: Sheet "MMR & Results", ubah 0.8 dan 0.2
4. **Vocabulary**: Sheet "c-TF-IDF", tambah/kurangi sample words

### Menambah Data:
1. Tambah baris di "Raw Data" dan "Embeddings"
2. Extend formula range di semua sheet
3. Update total dokumen (100) di formula IDF

## 🔍 INTERPRETASI HASIL

### Cluster Quality:
- Cluster -1 = outliers/noise
- Cluster 0,1,2,... = topik yang ditemukan
- Confidence > 0.5 = assignment yang baik

### Topic Quality:
- c-TF-IDF > 0.1 = kata penting untuk topik
- MMR rank 1-5 = kata kunci utama topik
- Similarity rendah = kata yang diverse

### Visualization:
- UMAP_X, UMAP_Y = koordinat 2D
- Titik yang dekat = dokumen serupa
- Warna berbeda = cluster berbeda

## ⚠️ LIMITASI

1. **Simplified Algorithms**: 
   - UMAP menggunakan PCA sederhana
   - HDBSCAN menggunakan density threshold

2. **Performance Limits**:
   - Max 100 dokumen (Excel performance)
   - Max 20 dimensi embedding

3. **Accuracy**:
   - ~70-80% akurasi vs BERTopic asli
   - Cocok untuk pembelajaran, bukan production

## 🚀 NEXT STEPS

1. **Analisis Hasil**: Review top words per cluster
2. **Validasi Manual**: Baca sample dokumen per cluster
3. **Tuning Parameter**: Adjust threshold sesuai data
4. **Visualisasi**: Buat scatter plot dari Visualization Data
5. **Scale Up**: Gunakan Python BERTopic untuk data besar

## 📞 TROUBLESHOOTING

**Formula Error**: Pastikan range data konsisten
**Slow Performance**: Kurangi jumlah dokumen/dimensi
**Wrong Results**: Check parameter di setiap sheet
**Missing Data**: Pastikan semua sheet terisi

---
Dibuat untuk pembelajaran BERTopic secara manual.
Untuk production use, gunakan library BERTopic Python.
"""
    
    with open('Manual_BERTopic_Guide.md', 'w', encoding='utf-8') as f:
        f.write(guide_content)
    
    print("✅ Usage guide created: Manual_BERTopic_Guide.md")

# Main execution
if __name__ == "__main__":
    print("🚀 Creating FULL MANUAL BERTopic Excel Calculator")
    print("=" * 60)
    
    try:
        # Create Excel file
        filename = create_manual_bertopic_excel()
        
        # Create usage guide
        create_usage_guide()
        
        print("\n✅ ALL FILES CREATED SUCCESSFULLY!")
        print("\n📋 FILES CREATED:")
        print("  1. BERTopic_Full_Manual_Calculation.xlsx")
        print("  2. Manual_BERTopic_Guide.md")
        
        print("\n🎯 KEY FEATURES:")
        print("  ✅ No Python required after file creation")
        print("  ✅ All calculations in Excel formulas")
        print("  ✅ Step-by-step manual process")
        print("  ✅ Educational and transparent")
        print("  ✅ Customizable parameters")
        
        print("\n⚠️ IMPORTANT NOTES:")
        print("  - Limited to 100 documents for performance")
        print("  - Simplified algorithms (70-80% accuracy)")
        print("  - Great for learning, not production")
        print("  - Excel may be slow with large calculations")
        
        print("\n🚀 NEXT STEPS:")
        print("  1. Open the Excel file")
        print("  2. Read the Instructions sheet")
        print("  3. Follow the calculation flow")
        print("  4. Analyze results in MMR & Results sheet")
        print("  5. Use Visualization Data for charts")
        
    except Exception as e:
        print(f"❌ Error creating files: {e}")
        import traceback
        traceback.print_exc()


🚀 Creating FULL MANUAL BERTopic Excel Calculator
Loading cleaned data...
Creating Instructions sheet...
Creating Raw Data sheet...
Creating Embeddings sheet...
Creating Distance Matrix sheet...
Creating Manual UMAP sheet...
Creating Manual HDBSCAN sheet...
Creating c-TF-IDF sheet...
Creating MMR & Results sheet...
Creating Visualization Data sheet...

✅ Excel file created: BERTopic_Full_Manual_Calculation.xlsx

📊 MANUAL EXCEL FILE SUMMARY:
  📁 Filename: BERTopic_Full_Manual_Calculation.xlsx
  📄 Total Sheets: 9
    1. Instructions
    2. Raw Data
    3. Embeddings
    4. Distance Matrix
    5. Manual UMAP
    6. Manual HDBSCAN
    7. c-TF-IDF
    8. MMR & Results
    9. Visualization Data
  🔢 Documents processed: 100 (limited for performance)
  📐 Embedding dimensions: 20 (limited for Excel)

🎯 CALCULATION FLOW:
  1. Raw Data → Embeddings
  2. Embeddings → Distance Matrix
  3. Distance Matrix → Manual UMAP
  4. Manual UMAP → Manual HDBSCAN
  5. Manual HDBSCAN → c-TF-IDF
  6. c-TF-IDF → M

# non duplicate data

🧹 PEMBERSIHAN DUPLIKASI REVIEW
🎯 Strategi: Hapus review duplikat, pertahankan multiple reviews per game
🔍 MENGHAPUS DUPLIKASI REVIEW (TETAP PERTAHANKAN MULTIPLE REVIEWS PER GAME)
📂 Loading data...
✅ Data berhasil dimuat: (1987, 772)

📊 STATISTIK SEBELUM PEMBERSIHAN:
  📈 Total baris: 1,987
  📈 Total kolom: 772
  🎮 Jumlah game unik: 100
  📝 Jumlah review unik: 361
  📈 Memory usage: 12.96 MB

🎮 DISTRIBUSI REVIEW PER GAME (SEBELUM):
  📊 Total games: 100
  📊 Rata-rata review per game: 19.9
  📊 Median review per game: 6.5
  📊 Game dengan review terbanyak: 100 reviews
  📊 Game dengan review tersedikit: 4 reviews

  🔝 TOP 10 GAMES (SEBELUM):
     1. Wolfenstein: The New Order: 100 reviews
     2. Warhammer 40,000: Space Marine 2: 100 reviews
     3. Battlefield™ V: 100 reviews
     4. Titanfall® 2: 100 reviews
     5. Team Fortress 2: 100 reviews
     6. Ready or Not: 100 reviews
     7. Life is Strange - Episode 1: 100 reviews
     8. Path of Exile: 100 reviews
     9. Outlast 2: 100 reviews


# Sheet

In [12]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill
import openpyxl

def create_manual_bertopic_excel():
    """
    Membuat file Excel dengan perhitungan UMAP dan HDBSCAN manual sesuai dengan instruksi.
    """
    
    # Load data cleaned
    print("Loading cleaned data...")
    # Menggunakan nama file yang benar yaitu 'embedding_no_dup.csv'
    df_embeddings = pd.read_csv('embeddings_no_dup.csv')  # Data embedding BERT
    df_preprocessed = pd.read_csv('preprocessed_Reviews.csv')  # Data review yang sudah diproses
    
    # Menghitung jumlah kata untuk setiap review dan menambahkannya sebagai kolom 'Num_Words'
    df_preprocessed['Num_Words'] = df_preprocessed['Review'].apply(lambda x: len(str(x).split()))
    
    # Hapus duplikasi berdasarkan 'row_index' di df_embeddings
    df_embeddings = df_embeddings.drop_duplicates(subset=['row_index'], keep='first')
    
    # Gabungkan df_embeddings dengan df_preprocessed berdasarkan 'row_index'
    df = pd.merge(df_preprocessed, df_embeddings[['row_index']], left_index=True, right_on='row_index', how='left')
    
    # Create workbook
    wb = Workbook()
    wb.remove(wb.active)
    
    # 1. SHEET "Instructions"
    print("Creating Instructions sheet...")
    ws_instr = wb.create_sheet("Instructions")
    
    instructions = [
        ["BERTopic FULL MANUAL Calculation in Excel", ""],
        ["", ""],
        ["🎯 OVERVIEW", "All calculations done in Excel - no Python required!"],
        ["", ""],
        ["📊 PROCESS FLOW:", ""],
        ["Step 1: Raw Data", "Original reviews and embeddings"],
        ["Step 2: Distance Matrix", "Euclidean distances between embeddings"],
        ["Step 3: Manual UMAP", "Simplified dimensionality reduction"],
        ["Step 4: Manual HDBSCAN", "Density-based clustering"],
        ["Step 5: c-TF-IDF", "Topic modeling calculations"],
        ["Step 6: MMR", "Keyword selection"],
        ["", ""],
        ["⚠️ LIMITATIONS:", ""],
        ["- Simplified UMAP (PCA-like approach)", ""],
        ["- Simplified HDBSCAN (k-means + density)", ""],
        ["- May not match exact BERTopic results", ""],
        ["- Good for understanding the process", ""],
        ["", ""],
        ["📈 EXPECTED PERFORMANCE:", ""],
        ["- Processing time: 5-10 minutes", ""],
        ["- Memory usage: High (distance matrices)", ""],
        ["- Accuracy: ~70-80% of full BERTopic", ""],
    ]
    
    for row_idx, (instruction, detail) in enumerate(instructions, 1):
        cell_a = ws_instr.cell(row=row_idx, column=1, value=instruction)
        cell_b = ws_instr.cell(row=row_idx, column=2, value=detail)
        
        if instruction and not instruction.startswith(" ") and not instruction.startswith("-"):
            cell_a.font = Font(bold=True, size=12)
            if any(x in instruction for x in ["Step", "🎯", "📊", "⚠️", "📈"]):
                cell_a.fill = PatternFill(start_color="D9E1F2", end_color="D9E1F2", fill_type="solid")
    
    ws_instr.column_dimensions['A'].width = 35
    ws_instr.column_dimensions['B'].width = 50
    
    # 2. SHEET "Raw Data"
    print("Creating Raw Data sheet...")
    ws_raw = wb.create_sheet("Raw Data")
    
    # Headers for Raw Data sheet
    headers_raw = ['row_index', 'Game', 'Review', 'Num_Words', 'Tokens', 'Case_Folding', 'Cleansing', 
                   'Normalization', 'Tokenized', 'Lemmatized', 'Final_Tokens', 'Cleaned_Reviews']
    
    for col, header in enumerate(headers_raw, 1):
        cell = ws_raw.cell(row=1, column=col, value=header)
        cell.font = Font(bold=True)
        cell.fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
        cell.font = Font(color="FFFFFF", bold=True)

    # Insert data from merged dataframe
    for row_idx, row in enumerate(df.itertuples(), 2):  # df sudah digabungkan
        ws_raw.cell(row=row_idx, column=1, value=row.row_index)  # row_index dari df_embeddings
        ws_raw.cell(row=row_idx, column=2, value=row.Game)   # Game
        ws_raw.cell(row=row_idx, column=3, value=row.Review) # Review
        ws_raw.cell(row=row_idx, column=4, value=row.Num_Words)  # Num_Words
        ws_raw.cell(row=row_idx, column=5, value=row.tokenized)     # Ganti 'Tokens' dengan 'tokenized'
        ws_raw.cell(row=row_idx, column=6, value=row.case_folding)   # Case_Folding
        ws_raw.cell(row=row_idx, column=7, value=row.cleansing)      # Cleasing
        ws_raw.cell(row=row_idx, column=8, value=row.normalization)  # Normalization
        ws_raw.cell(row=row_idx, column=9, value=row.tokenized)      # Tokenized
        ws_raw.cell(row=row_idx, column=10, value=row.lemmatized)    # Lemmatized
        ws_raw.cell(row=row_idx, column=11, value=row.final_tokens)  # Final_Tokens
        ws_raw.cell(row=row_idx, column=12, value=row.cleaned_Reviews)  # Perbaiki menjadi 'cleaned_Reviews'
    
    # 3. SHEET "Embedding"
    print("Creating Embedding sheet...")
    ws_embed = wb.create_sheet("Embedding")
    
    embedding_cols = [col for col in df_embeddings.columns if col.startswith('embedding_')]
    embed_headers = ['Token'] + [f'E{i}' for i in range(len(embedding_cols))]
    
    for col, header in enumerate(embed_headers, 1):
        cell = ws_embed.cell(row=1, column=col, value=header)
        cell.font = Font(bold=True)
        cell.fill = PatternFill(start_color="70AD47", end_color="70AD47", fill_type="solid")
        cell.font = Font(color="FFFFFF", bold=True)
    
    # Add embedding data
    for row_idx in range(len(df_embeddings)):  # Use all data from the file
        ws_embed.cell(row=row_idx + 2, column=1, value=df_embeddings.iloc[row_idx]['row_index'])
        for col_idx, embed_col in enumerate(embedding_cols, 2):
            ws_embed.cell(row=row_idx + 2, column=col_idx, value=df_embeddings.iloc[row_idx][embed_col])
    
    # 4. SHEET "UMAP"
    print("Creating UMAP sheet...")
    ws_umap = wb.create_sheet("UMAP")
    
    umap_headers = ['Token', 'UMAP Dim1', 'UMAP Dim2']
    for col, header in enumerate(umap_headers, 1):
        cell = ws_umap.cell(row=1, column=col, value=header)
        cell.font = Font(bold=True)
        cell.fill = PatternFill(start_color="FFC000", end_color="FFC000", fill_type="solid")
        cell.font = Font(color="000000", bold=True)
    
    # Insert UMAP calculations (manual or from Python)
    for row_idx in range(2, len(df) + 2):
        ws_umap.cell(row=row_idx, column=1, value=f"=Embedding!A{row_idx}")
        ws_umap.cell(row=row_idx, column=2, value=f"=AVERAGE(Embedding!B{row_idx}:K{row_idx})")
        ws_umap.cell(row=row_idx, column=3, value=f"=AVERAGE(Embedding!L{row_idx}:U{row_idx})")
    
    # 5. SHEET "HDBSCAN"
    print("Creating HDBSCAN sheet...")
    ws_hdbscan = wb.create_sheet("HDBSCAN")
    
    hdbscan_headers = ['Token', 'Cluster', 'Probability']
    for col, header in enumerate(hdbscan_headers, 1):
        cell = ws_hdbscan.cell(row=1, column=col, value=header)
        cell.font = Font(bold=True)
        cell.fill = PatternFill(start_color="FF99CC", end_color="FF99CC", fill_type="solid")
        cell.font = Font(color="000000", bold=True)
    
    # Placeholder for HDBSCAN output
    for row_idx in range(2, len(df) + 2):
        ws_hdbscan.cell(row=row_idx, column=1, value=f"=UMAP!A{row_idx}")
        ws_hdbscan.cell(row=row_idx, column=2, value=f"=RANDBETWEEN(0, 10)")  # Simulated cluster labels
        ws_hdbscan.cell(row=row_idx, column=3, value=f"=RAND()")  # Simulated probabilities
    
    # 6. SHEET "c-TF-IDF"
    print("Creating c-TF-IDF sheet...")
    ws_ctfidf = wb.create_sheet("c-TF-IDF")
    
    ctfidf_headers = ['Cluster ID', 'Total Dokumen Cluster', 'Kata', 'Frekuensi per Cluster', 'TF', 'DF', 'IDF', 'c-TF-IDF']
    for col, header in enumerate(ctfidf_headers, 1):
        cell = ws_ctfidf.cell(row=1, column=col, value=header)
        cell.font = Font(bold=True)
        cell.fill = PatternFill(start_color="CCCCFF", end_color="CCCCFF", fill_type="solid")
        cell.font = Font(color="000000", bold=True)
    
    # Placeholder for c-TF-IDF calculations (simulated)
    for row_idx in range(2, len(df) + 2):
        ws_ctfidf.cell(row=row_idx, column=1, value=f"=HDBSCAN!B{row_idx}")
        ws_ctfidf.cell(row=row_idx, column=2, value=f"=COUNTIF(HDBSCAN!$B$2:$B$370, A{row_idx})")
        ws_ctfidf.cell(row=row_idx, column=3, value=f"=Embedding!A{row_idx}")  # Simulated kata

    # Save workbook
    filename = "BERTopic_Manual_Calculation.xlsx"
    wb.save(filename)
    print(f"\n✅ Excel file created: {filename}")
    
    return filename

# Main execution
if __name__ == "__main__":
    filename = create_manual_bertopic_excel()
    print(f"File created: {filename}")


Loading cleaned data...
Creating Instructions sheet...
Creating Raw Data sheet...
Creating Embedding sheet...
Creating UMAP sheet...
Creating HDBSCAN sheet...
Creating c-TF-IDF sheet...

✅ Excel file created: BERTopic_Manual_Calculation.xlsx
File created: BERTopic_Manual_Calculation.xlsx


In [16]:
df_preprocessed = pd.read_csv("preprocessed_Reviews.csv")
print(df_preprocessed.columns)

Index(['Game', 'Review', 'case_folding', 'cleansing', 'normalization',
       'tokenized', 'lemmatized', 'final_tokens', 'cleaned_Reviews'],
      dtype='object')


In [3]:
df = pd.read_csv("sentence_embeddings_no_duplicate_reviews.csv")
print(df.columns)

Index(['row_index', 'game', 'review', 'num_words', 'embedding_0',
       'embedding_1', 'embedding_2', 'embedding_3', 'embedding_4',
       'embedding_5',
       ...
       'embedding_758', 'embedding_759', 'embedding_760', 'embedding_761',
       'embedding_762', 'embedding_763', 'embedding_764', 'embedding_765',
       'embedding_766', 'embedding_767'],
      dtype='object', length=772)


In [14]:
import pandas as pd

# Baca file CSV
df_embeddings = pd.read_csv('embedding_bertopic.csv')

# Simpan data ke file Excel
df_embeddings.to_excel('embedding_bertopic.xlsx', index=False)

print("File CSV berhasil diubah menjadi Excel.")

File CSV berhasil diubah menjadi Excel.
